In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    AutoModel
)

from tabulate import tabulate
import nltk
from datetime import datetime

2022-11-26 16:05:42.041874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1,load model, tokenizer 

In [16]:
import pandas as pd
data = pd.read_csv('clean_covid.csv')

/var/folders/nh/wmbw_rv57_g2jq_w3v99jyrr0000gn/T/ipykernel_52413/1189849771.py:2: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('clean_covid.csv')


In [22]:
data[['abstract','title']].dropna()

,abstract,title
0,objective this retrospective chart review desc...,clinical features of culture proven mycoplasma...
1,inflammatory diseases of the respiratory tract...,nitric oxide a pro inflammatory mediator in lu...
2,surfactant protein d sp d participates in the ...,surfactant protein d and pulmonary host defense
3,endothelin et is a amino acid peptide with div...,role of endothelin in lung disease
4,respiratory syncytial virus rsv and pneumonia ...,gene expression in epithelial cells in respons...
...,...,...
814829,ncov which is a novel coronavirus emerged in ...,potent neutralization of novel coronavirus by ...
814830,microbiology laboratories have traditionally r...,molecular based diagnostics including future t...
814831,to present a patient with bilateral conjunctiv...,a patient with bilateral conjunctivitis positi...
814832,given covid pandemic periodic outpatient asses...,incidental lowering of otitis media complaints...


In [23]:
data[['abstract','title']]

,abstract,title
0,objective this retrospective chart review desc...,clinical features of culture proven mycoplasma...
1,inflammatory diseases of the respiratory tract...,nitric oxide a pro inflammatory mediator in lu...
2,surfactant protein d sp d participates in the ...,surfactant protein d and pulmonary host defense
3,endothelin et is a amino acid peptide with div...,role of endothelin in lung disease
4,respiratory syncytial virus rsv and pneumonia ...,gene expression in epithelial cells in respons...
...,...,...
814829,ncov which is a novel coronavirus emerged in ...,potent neutralization of novel coronavirus by ...
814830,microbiology laboratories have traditionally r...,molecular based diagnostics including future t...
814831,to present a patient with bilateral conjunctiv...,a patient with bilateral conjunctivitis positi...
814832,given covid pandemic periodic outpatient asses...,incidental lowering of otitis media complaints...


In [12]:
data['abstract'].astype('string')
data['abstract'].astype('string')

0         objective this retrospective chart review desc...
1         inflammatory diseases of the respiratory tract...
2         surfactant protein d sp d participates in the ...
3         endothelin et is a amino acid peptide with div...
4         respiratory syncytial virus rsv and pneumonia ...
                                ...                        
814829     ncov which is a novel coronavirus emerged in ...
814830    microbiology laboratories have traditionally r...
814831    to present a patient with bilateral conjunctiv...
814832    given covid pandemic periodic outpatient asses...
814833    objective to examine interhospital variation i...
Name: abstract, Length: 814834, dtype: string

In [14]:
data['abstract'] = data['abstract'].astype('string')

In [15]:
data['abstract']

0         objective this retrospective chart review desc...
1         inflammatory diseases of the respiratory tract...
2         surfactant protein d sp d participates in the ...
3         endothelin et is a amino acid peptide with div...
4         respiratory syncytial virus rsv and pneumonia ...
                                ...                        
814829     ncov which is a novel coronavirus emerged in ...
814830    microbiology laboratories have traditionally r...
814831    to present a patient with bilateral conjunctiv...
814832    given covid pandemic periodic outpatient asses...
814833    objective to examine interhospital variation i...
Name: abstract, Length: 814834, dtype: string

In [102]:
model_name = 'Chikashi/t5-small-finetuned-cnndm'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file config.json from cache at /Users/kaiyuhe/.cache/huggingface/hub/models--Chikashi--t5-small-finetuned-cnndm/snapshots/13cf730d52abe6030eb376fd9156ea6474da5448/config.json
Model config T5Config {
  "_name_or_path": "Chikashi/t5-small-finetuned-cnndm",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,


# 2, load and preprocess dataset  and fine tune model
- create dataset using pandas
- using tokenizer preprocess dataset
- finetune model
- save finetuned model

In [222]:
from datasets import Dataset
asd = {'article':['abd asd', 'asdasff asfasdas'],
      'highlights':['asdasd','asdasd']}
Dataset.from_dict(asd)

In [225]:
# create or load dataset and the clean dataset 
# onlt two features each sample ['document','summary']
dataset = datasets.load_dataset('cnn_dailymail','1.0.0')
def flatten_data(example):
    return {'document': example['article'],'summary': example['highlights']}
def list2samples(example):
    documents = []
    summaries = []
    for sample in zip(example["article"], example["summary"]):
        if len(sample[0]) > 0:
            documents += sample[0]
            summaries += sample[1]
    return {"document": documents, "summary": summaries}
dataset = dataset['validation'].map(flatten_data, remove_columns=["id",'highlights','article'])
#dataset = dataset.map(list2samples, batched=True)
train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()
encoder_max_length = 512  # demo
decoder_max_length = 100

Found cached dataset cnn_dailymail (/Users/kaiyuhe/.cache/huggingface/datasets/cnn_dailymail/1.0.0/1.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/kaiyuhe/.cache/huggingface/datasets/cnn_dailymail/1.0.0/1.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-8c95654dd7312e80.arrow


In [5]:
# tokenize examples
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["document"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
# only select 1000 examples for demo
train_data = train_data.select(range(1000))

In [7]:
# metrics for summarization
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

/var/folders/nh/wmbw_rv57_g2jq_w3v99jyrr0000gn/T/ipykernel_35386/692090678.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


In [8]:
# start finetune the model
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=512,  # demo
    per_device_eval_batch_size=512,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/Users/kaiyuhe/opt/anaconda3/envs/huggingface/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text follow

Step,Training Loss
50,3.905300
100,3.860800
150,3.823400
200,3.713500
250,3.624100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=250, training_loss=3.785421325683594, metrics={'train_runtime': 706.1081, 'train_samples_per_second': 1.416, 'train_steps_per_second': 0.354, 'total_flos': 135341801472000.0, 'train_loss': 3.785421325683594, 'epoch': 1.0})

In [9]:
#save model
trainer.save_model('demotuned_model')

Saving model checkpoint to demotuned_model
Configuration saved in demotuned_model/config.json
Model weights saved in demotuned_model/pytorch_model.bin
tokenizer config file saved in demotuned_model/tokenizer_config.json
Special tokens file saved in demotuned_model/special_tokens_map.json


# 3, Use finetuned model generate candidate summaries

In [103]:
# load model
# candidate parameter
model = AutoModelForSeq2SeqLM.from_pretrained('demotuned_model')

loading configuration file demotuned_model/config.json
Model config T5Config {
  "_name_or_path": "demotuned_model",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_s

In [104]:
document = dataset['document']
reference = dataset['summary']

In [105]:
input_ids = tokenizer(document,truncation=True, 
                      padding = True, max_length = 512).input_ids
input_ids = torch.tensor(input_ids, dtype=torch.long)

In [106]:
input_ids.shape

torch.Size([13368, 512])

In [4]:
a = 1
b = 2
f'{a}++{b}'

'1++2'

In [107]:
input_ids = input_ids[:10,:]

In [108]:
input_ids.shape

torch.Size([10, 512])

### parameters need to specifiy
- decoder seq length: max_length
- diversity_penalty

In [109]:
# diverse beam search
num_return_seqs = 16
outputs = model.generate(input_ids,num_beams = num_return_seqs,
                         #no_repeat_ngram_size=2,
                         diversity_penalty=1.0,
                         max_length = 100,
                         num_beam_groups = num_return_seqs,
    num_return_sequences=num_return_seqs)

/Users/kaiyuhe/opt/anaconda3/envs/huggingface/lib/python3.10/site-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


In [57]:
outputs[16:,:].shape

torch.Size([144, 86])

In [115]:
test_str = tokenizer.decode(outputs[0],skip_special_tokens=True)

In [191]:
tokenizer_try = AutoTokenizer.from_pretrained('gpt2')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/kaiyuhe/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
  

In [192]:
tokenizer_try.tokenize('I really love to play')

['I', 'Ġreally', 'Ġlove', 'Ġto', 'Ġplay']

In [190]:
tokenizer_try.tokenize('I really love to play')

['i', 'really', 'love', 'to', 'play']

In [186]:
tokenizer.tokenize('A B')[1][0]

'▁'

In [156]:
tokenizer.decode([3, 834, 1])

'_</s>'

In [129]:
''.join(tokenizer.tokenize(test_str)).replace('▁',' ')

' Zully Broussard gave one of her kidneys to a stranger, her generosity paired up with big data. The power multiplied the gift was data processing of genetic profiles from donor-recipient pairs. The chain of surgeries is to be wrapped up Friday.'

### Start write files

In [194]:
document = document[:16]
reference = reference[:16]

In [218]:
def write_data_for_preprocess(tran_test_val, save_dir,doc,refer,cand):
    os.makedirs(save_dir)
    cand = [line+'\n' for line in cand]
    doc = [line+'\n' for line in doc]
    refer = [line+'\n' for line in refer]
    with open(f'{save_dir}/{tran_test_val}.out','w') as handle:
        handle.writelines(cand)
    with open(f'{save_dir}/{tran_test_val}.out.tokenized','w') as handle:
        handle.writelines(cand)
    with open(f'{save_dir}/{tran_test_val}.source','w') as handle:
        handle.writelines(doc)
    with open(f'{save_dir}/{tran_test_val}.source.tokenized','w') as handle:
        handle.writelines(doc)
    with open(f'{save_dir}/{tran_test_val}.target','w') as handle:
        handle.writelines(refer)
    with open(f'{save_dir}/{tran_test_val}.target.tokenized','w') as handle:
        handle.writelines(refer)

In [219]:
write_data_for_preprocess('train','first_write',document, reference, cand_summaries)

In [220]:
!python preprocess.py --src_dir first_write --tgt_dir targetdataset --split train --cand_num 16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
finish


In [ ]:
if os.p

In [ ]:
# sampling search
num_return_seqs = 16
outputs = model.generate(input_ids, do_sample=False, num_beams = num_return_seqs,
    top_k=50, 
    top_p=0.95, 
                         temperature=0.6,
                         no_repeat_ngram_size=2,
                         diversity_penalty=1.0,
                         num_beam_groups = num_return_seqs,
    num_return_sequences=num_return_seqs)

# 4, Train SimCLS scorer

# 5, Evaluate